# **This Notebook is provided Training Example (PhC-C2DH-U373 Dataset) Using Colab**
The notebook arragemnet is as follow

*   **Prequesities**
  1.   Conect to github, Clone the repository
  2.   Configurate working directory and google drive connetction
  3.   Download and install Conda, create conda enviroment with all the required packages

*   **Trainings**
  1.   Run feature extraction for metric learning
  2.   Run metric learning training
  3.   Run compreensive feature extraction with metric learning
  4.   Run Cell Tracking training

*   **Inference**








# **Prequesities**

# 1. Conect to github, Clone the repository


In [ ]:
import os
! git clone https://github.com/talbenha/cell-tracker-gnn

curr_workdir = f'./cell-tracker-gnn'
if os.path.exists(curr_workdir):
  os.chdir(curr_workdir)

Cloning into 'cell-tracker-gnn'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 323 (delta 74), reused 35 (delta 35), pack-reused 229
Receiving objects: 100% (323/323), 947.33 KiB | 12.98 MiB/s, done.
Resolving deltas: 100% (160/160), done.


# 2. Configurate working directory and google drive connetction


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/CellTracking/nature_method_ctc/' # TODO: update this path
os.listdir(file_path)

Mounted at /content/drive


['AC (5) Description.gdoc',
 'Fluo-N2DH-SIM+',
 'csv',
 'basic_features',
 'PhC-C2DH-U373',
 'software',
 'ct_features',
 'training_example.ipynb',
 'inference_example.ipynb']

# 3. Download and install Conda, create conda enviroment with all the required packages

In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################

# try to run the bare minimum to get a new conda env working
env_name = 'cell-tracking-challenge'
conda_path = ''
try:
    conda_path = ! which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
    import sys
    sys.path.append('/usr/local/lib/python3.7/site-packages/')
    ! conda init
    ! conda install -n root _license -y -q
else:
    print('found miniconda')

conda_envs = ! conda env list
res = [i for i in conda_envs if env_name in i]
if (len(res) == 0):
    print(f'not found {env_name} env', len(res))
    ! conda create --name cell-tracking-challenge --file requirements-conda.txt
else:
    print(f'found {env_name} env', len(res))
print(f'{env_name} was created')


installing miniconda
--2022-07-16 19:19:14--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  18.6MB/s    in 4.4s    

2022-07-16 19:19:18 (18.6 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-packag

In [ ]:
%%bash
source activate cell-tracking-challenge
# install other requirements
pip install -r requirements.txt
pip install PyYAML==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=61ed0fa6fb5af19d82c8be44d78c1476f00dfecfddec547f2df11b9515e1f7a2
  Stored in directory: /root/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=3475c7e6f417ee36dddbd77d7dc177cd6926ba513aae3e95c23ee2d085a986cb
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ea21a91ae6e1f3596ad3b337af8c3c70b1b96d523c796a5414159d98aa6e4388
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=82e

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-geometric 2.0.1 requires rdflib, which is not installed.


# **Training**
 The first two stages can be skipped (step #2 is long in colab) by using the pre-trained model (in step #3) provided under the [releases](https://github.com/talbenha/cell-tracker-gnn/releases) of the repository. For example for the pretrained model for PhC-C2DH-U373 dataset, download the software and find the model under `software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth`

# 1. Run feature extraction for metric learning


In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path

python run_feat_extract.py params.input_images="${FOLDER_PATH}/${DATA_NAME}" params.input_masks="${FOLDER_PATH}/${DATA_NAME}" params.input_seg="${FOLDER_PATH}/${DATA_NAME}" params.output_csv="${FOLDER_PATH}/basic_features" params.sequences=['01','02']  params.seg_dir='_GT/TRA' params.basic=True

[2022-07-16 19:31:50,897][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
files were saved to : /content/drive/MyDrive/CellTracking/nature_method_ctc/basic_features/PhC-C2DH-U373/01/csv
files were saved to : /content/drive/MyDrive/CellTracking/nature_method_ctc/basic_features/PhC-C2DH-U373/02/csv


# 2. Run metric learning training
 Pay attention that this cell takes long time on google colab, so you can skip this stage and use the pre-trained model provided under the [releases](https://github.com/talbenha/cell-tracker-gnn/releases) of the repository. For example for the pretrained model for PhC-C2DH-U373 dataset, download the software and find the model under `software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth`

In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path

python run_train_metric_learning.py dataset.kwargs.data_dir_img="${FOLDER_PATH}/${DATA_NAME}" dataset.kwargs.data_dir_mask="${FOLDER_PATH}/${DATA_NAME}" dataset.kwargs.dir_csv="${FOLDER_PATH}/basic_features/${DATA_NAME}" dataset.kwargs.subdir_mask='GT/TRA'

--> output 'all_params.pth' is generated at end, it is the input_model (i.e. for METRIC_PATH variable) for the next step

# 3. Run compreensive feature extraction with metric learning

In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export METRIC_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc/software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path

python run_feat_extract.py params.input_images="${FOLDER_PATH}/${DATA_NAME}" params.input_masks="${FOLDER_PATH}/${DATA_NAME}" params.input_seg="${FOLDER_PATH}/${DATA_NAME}" params.output_csv="${FOLDER_PATH}/ct_features/" params.sequences=['01','02']  params.seg_dir='_GT/TRA' params.basic=False params.input_model="${METRIC_PATH}"

[2022-07-16 19:34:05,951][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
Using resnet18 model architecture.
files were saved to : /content/drive/MyDrive/CellTracking/nature_method_ctc/ct_features/PhC-C2DH-U373/01/csv
Using resnet18 model architecture.
files were saved to : /content/drive/MyDrive/CellTracking/nature_method_ctc/ct_features/PhC-C2DH-U373/02/csv


# 4. Run Cell Tracking training

In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path
# cell tracking training run
python run.py datamodule.dataset_params.main_path="${FOLDER_PATH}/ct_features/${DATA_NAME}" datamodule.dataset_params.exp_name="2D_SIM" datamodule.dataset_params.drop_feat=[]

[2022-07-16 19:37:10,310][src.utils.utils][INFO] - Disabling python warnings! <config.disable_warnings=True>
⚙ CONFIG                                                                        
├── trainer                                                                     
│   └── _target_: pytorch_lightning.Trainer                                     
│       gpus: 1                                                                 
│       min_epochs: 1                                                           
│       max_epochs: 500                                                         
│       weights_summary: top                                                    
│       progress_bar_refresh_rate: 100                                          
│       terminate_on_nan: true                                                  
│                                                                               
├── model                                                                       


# **Inference**
Update the following variables:


*   MODEL_PYTORCH_LIGHTNING - according to the output of the previius cell
*   MODEL_METRIC_LEARNING
*   FOLDER_PATH - path to folder that contains the dataset
*   MODALITY





In [ ]:
%%bash
source activate cell-tracking-challenge
export SEQUENCE=01 # sequence number
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export MODEL_METRIC_LEARNING=/content/drive/MyDrive/CellTracking/nature_method_ctc/software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth # TODO: update this path
export MODEL_PYTORCH_LIGHTNING='/content/cell-tracker-gnn/logs/runs/2022-07-16/19-37-07/checkpoints/epoch=37.ckpt'
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path
export MODALITY="2D"  # dataset modality

export DATASET="${FOLDER_PATH}/${DATA_NAME}"
export CODE_TRA=src/inference/
export PYTHONPATH=/content/cell-tracker-gnn

# our model needs CSVs, so let's create from image and segmentation.
python ${CODE_TRA}/preprocess_seq2graph_clean.py -cs 20 -ii "${DATASET}/${SEQUENCE}" -iseg "${DATASET}/${SEQUENCE}_GT/TRA" -im "${MODEL_METRIC_LEARNING}" -oc "${DATASET}/${SEQUENCE}_CSV"

# run the prediction
python ${CODE_TRA}/inference_clean.py -mp "${MODEL_PYTORCH_LIGHTNING}" -ns "${SEQUENCE}" -oc "${DATASET}"

# create label maps
python ${CODE_TRA}/postprocess_clean.py -modality "${MODALITY}" -iseg "${DATASET}/${SEQUENCE}_GT/TRA" -oi "${DATASET}/${SEQUENCE}_RES_inference"


number of detected changes: 0
0
global_delta_row: 46
global_delta_col: 37
We don't assign new region of interest - use the old one
Using resnet18 model architecture.
files were saved to : /content/drive/MyDrive/CellTracking/nature_method_ctc/PhC-C2DH-U373/01_CSV/csv
load model from: /content/cell-tracker-gnn/logs/runs/2022-07-16/19-37-07/checkpoints/epoch=37.ckpt
"directed":        True
"loss_weights":    65
"lr":              0.001
"lr_sch_module":   {'target': None, 'monitor': 'val/acc', 'kwargs': {'verbose': True, 'mode': 'min'}}
"model_params":    {'target': 'CellTrack_Model', 'kwargs': {'hand_NodeEncoder_dic': {'input_dim': 13, 'fc_dims': [64, 16]}, 'learned_NodeEncoder_dic': {'input_dim': 128, 'fc_dims': [64, 16]}, 'intialize_EdgeEncoder_dic': {'input_dim': 239, 'fc_dims': [128, 64]}, 'message_passing': {'target': 'CellTrack_GNN', 'kwargs': {'in_channels': 32, 'hidden_channels': 32, 'in_edge_channels': 64, 'hidden_edge_channels_conv': 16, 'hidden_edge_channels_linear': [128, 64],